<a href="https://colab.research.google.com/github/Location-Artistry/ColabNotebooks/blob/master/paDataPyMap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Working Notebook for visualizing in GeoPandas
Edited Sep 7th - GeoPandas can directly load GeoJSON from url, does not work with regular JSON   
Working to flatten/normalize nested json with Pandas/GeoPandas   
Additionally bring in ArcGIS Feature Service as DataFrame/GeoDataFrame

In [ ]:
%%time 
!apt update
!apt upgrade
# Important library for many geopython libraries
!apt install gdal-bin python-gdal python3-gdal 
# Install rtree - Geopandas requirment
!apt install python3-rtree 
# Install Geopandas
!pip install git+git://github.com/geopandas/geopandas.git
# Install descartes - Geopandas requirment
!pip install descartes 
# Install Folium for Geographic data visualization
!pip install folium
!pip install pygeos
!pip install flatten_json 

In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
import matplotlib
import matplotlib.pyplot as plt 
import folium
import os
from IPython.display import display
from pandas.io.json import json_normalize
from flatten_json import flatten 
import requests

/usr/local/lib/python3.6/dist-packages/geopandas/_compat.py:87: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.8.1-CAPI-1.13.3). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


#Start Working Section

In [ ]:
def GeoJSONtoGDF(jsonURL):
  geoDF = gpd.read_file(jsonURL)
  display(geoDF.head())
  return geoDF

In [ ]:
NHBPpa = 'https://purpleairwidget.firebaseapp.com/purpleAirData/44439,2822,5712,43297,7772,37659'
PAallSta = 'https://www.purpleair.com/json'
miCounties = 'https://opendata.arcgis.com/datasets/67a8ff23b5f54f15b7133b8c30981441_0.geojson'

gdf = GeoJSONtoGDF(NHBPpa)
gdf2 = GeoJSONtoGDF(miCounties)
#gdf3 = GeoJSONtoGDF(PAallSta)

In [ ]:
# Overlay points and Polygons
fig, ax = plt.subplots(figsize=(10,10))
gdf2.plot(ax=ax, column='NAME');
#ax.margins(-.2, -.2)
ax.set_xticklabels([])
ax.set_yticklabels([])
gdf.plot(ax=ax, markersize=100, color='black');

In [ ]:
import json 

response = requests.get(NHBPpa)
rText = response.text
rText
res = json.loads(rText) 
res2 = pd.json_normalize(res, max_level=2)
data = json.loads(rText)
df = pd.json_normalize(data['features'])
df['geometry'] = df['geometry.coordinates']
#df['lat'] = df.apply(lambda row: row['geometry.coordinates'])
#df
gdf = gpd.read_file(df)
#dfEX['SITE #'] = dfEX.apply(lambda row: siteList[row.SITE_ID], axis=1)

In [ ]:
for lat, lon in zip((df['geometry.coordinates'][0])[:5], df['geometry.coordinates'][:5]):
    geometry = lat, lon
    geomPts = []
    geomPts.append(geometry)
    print(geometry)

(-85.269179, [-85.269179, 42.102401])
(42.102401, [-86.17218, 41.76787])


In [ ]:
import json 

response = requests.get(NHBPpa)
rText = response.text
rText
res = json.loads(rText) 
res2 = pd.json_normalize(res, max_level=2)
data = json.loads(rText)
df = pd.json_normalize(data['features'])
df['geometry.coordinates'][0][1]
#gdf = gpd.read_file(df)


42.102401

In [ ]:

request = request(NHBPpa)
response = urlopen(request)
stations = response.read()
data = json.loads(stations)
df = pd.json_normalize(data['features'])